# 地理データの解析

## 1. 大阪の平均路線価の可視化

利用するデータについて。

大阪の中心部の平均路線価に関するデータを用います。[ここ](https://data.city.osaka.lg.jp/data/dataset/data-00000065)からダウンロードできる`tikatyousa_2018_100m_souzokurosenka.csv`というファイルで、「地価調査\_平成30年\_100ｍ\_相続税路線価\_データベースファイル（CSV）」という項目をクリックしてダウンロードページへ行き、ダウンロードをしてください。

データセットの概要は、地価データを用いて、大阪市内の最高路線価、最低路線価等について調査したものです。路線価というのは相続税の計算の際に使われる土地の価格のことです。この解析では「平均路線価」を使いますが、ここで言う「平均」というのは、ある区画内にある建物の路線価を平均したということです。

今回利用する環境(binder)について。

binderとは、「GitHub の repository から(使い捨ての)Jupyter Notebook の環境を構築しその(計算)環境を誰もが利用できる形で提供してくれるウェブアプリケーション」([西田さんのQiitaの記事](https://qiita.com/kozo2/items/fef4ee9c175f941a8a59)より引用)のことです。
詳しくは、本チュートリアルの講師の一人である[西田さんが書かれた記事](https://qiita.com/kozo2/items/fef4ee9c175f941a8a59)を参考にしてください。

では、解析を始めましょう。

まず、今回利用するモジュールをimportします。

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pydeck as pdk

また、mapboxを使うためのAPIキーが必要なので、ここで各々のキーの値を入力してください。

In [2]:
import getpass

print('mapboxのapikeyを入力してください')
MAPBOX_KEY = getpass.getpass()

mapboxのapikeyを入力してください


 ·····················································································


では、まず最初に、geopandasでshapeファイルを読み込みます。

In [3]:
data = gpd.read_file('../data/osakadata/tikatyousa_2018_100m_souzokurosenka.shp', encoding='shift-jis')

データの先頭は以下のようになっています。

In [4]:
data.head()

,MESH_CODE,最高路線価,最低路線価,平均路線価,geometry
0,S115E07025,0,0,0,"POLYGON ((-38261.250 -157053.172, -38261.250 -..."
1,S115E07024,0,0,0,"POLYGON ((-38361.249 -157053.172, -38361.249 -..."
2,S115E07023,0,0,0,"POLYGON ((-38461.248 -157053.171, -38461.248 -..."
3,S115E07022,0,0,0,"POLYGON ((-38561.247 -157053.171, -38561.247 -..."
4,S115E07021,0,0,0,"POLYGON ((-38661.245 -157053.171, -38661.246 -..."


crs属性を見ることで、読み込んだShapeの座標系等を確認することができます。

In [5]:
data.crs

<Projected CRS: EPSG:2448>
Name: JGD2000 / Japan Plane Rectangular CS VI
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Japan - zone VI
- bounds: (134.86, 33.4, 136.99, 36.33)
Coordinate Operation:
- name: Japan Plane Rectangular CS zone VI
- method: Transverse Mercator
Datum: Japanese Geodetic Datum 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

上記の結果から、EPSGが2448であることがわかります。EPSGはEuropean Petroleum Survey Groupという団体によって作成された、測地系、座標系、地図投影法等をコード体系にしたものです。EPSGが2448ということは、今、平面直角座標系なので、GPSで得られる位置・緯度経度に変換するために、EPSGを4326に設定します。

In [6]:
data = data.to_crs(epsg=4326)

In [7]:
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

読み込んだデータの大まかな形(記述統計量)を見るために、describe()を使ってdataの統計量を確認してみましょう。

In [8]:
data.describe()

,最高路線価,最低路線価,平均路線価
count,32700.000000,32700.000000,32700.000000
mean,128.590214,73.553792,101.202630
std,378.689611,162.750181,256.749798
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,81.000000,56.000000,71.000000
75%,152.000000,103.000000,129.000000
max,12110.000000,11561.000000,11561.000000


この結果から、データの中のそれぞれの価格が0のことが多いことがわかります。

.centroidを使うことで、Polygonの中心の緯度・経度を得ることができます。

In [9]:
data['geometry'].centroid

/home/ksn/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


0        POINT (135.58239 34.58311)
1        POINT (135.58130 34.58311)
2        POINT (135.58021 34.58310)
3        POINT (135.57912 34.58310)
4        POINT (135.57803 34.58310)
5        POINT (135.57694 34.58309)
6        POINT (135.57585 34.58309)
7        POINT (135.57476 34.58308)
8        POINT (135.57367 34.58308)
9        POINT (135.57258 34.58308)
10       POINT (135.57149 34.58307)
11       POINT (135.57040 34.58307)
12       POINT (135.56931 34.58307)
13       POINT (135.56822 34.58306)
14       POINT (135.56713 34.58306)
15       POINT (135.56604 34.58305)
16       POINT (135.56495 34.58305)
17       POINT (135.56386 34.58305)
18       POINT (135.56277 34.58304)
19       POINT (135.56168 34.58304)
20       POINT (135.56059 34.58303)
21       POINT (135.55950 34.58303)
22       POINT (135.55841 34.58303)
23       POINT (135.55732 34.58302)
24       POINT (135.55623 34.58302)
25       POINT (135.55514 34.58301)
26       POINT (135.55405 34.58301)
27       POINT (135.55296 34

.centroid.xで経度が得られ、.centroid.yで緯度が得られます。

In [10]:
data['geometry'].centroid.x

/home/ksn/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


0        135.582393
1        135.581303
2        135.580213
3        135.579123
4        135.578033
5        135.576943
6        135.575853
7        135.574763
8        135.573673
9        135.572583
10       135.571493
11       135.570403
12       135.569313
13       135.568223
14       135.567133
15       135.566043
16       135.564953
17       135.563863
18       135.562773
19       135.561683
20       135.560593
21       135.559503
22       135.558413
23       135.557323
24       135.556233
25       135.555143
26       135.554053
27       135.552963
28       135.551873
29       135.550783
            ...    
32670    135.525732
32671    135.524640
32672    135.537749
32673    135.523547
32674    135.522455
32675    135.538842
32676    135.521362
32677    135.539934
32678    135.520270
32679    135.541027
32680    135.519177
32681    135.542119
32682    135.518085
32683    135.543211
32684    135.516993
32685    135.544304
32686    135.545396
32687    135.546489
32688    135.547581


In [11]:
data['geometry'].centroid.y

/home/ksn/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


0        34.583111
1        34.583107
2        34.583103
3        34.583099
4        34.583096
5        34.583092
6        34.583088
7        34.583084
8        34.583080
9        34.583077
10       34.583073
11       34.583069
12       34.583065
13       34.583061
14       34.583057
15       34.583054
16       34.583050
17       34.583046
18       34.583042
19       34.583038
20       34.583034
21       34.583030
22       34.583026
23       34.583022
24       34.583018
25       34.583014
26       34.583010
27       34.583006
28       34.583002
29       34.582998
           ...    
32670    34.771318
32671    34.771314
32672    34.771364
32673    34.771310
32674    34.771305
32675    34.771368
32676    34.771301
32677    34.771373
32678    34.771297
32679    34.771377
32680    34.771292
32681    34.771381
32682    34.771288
32683    34.771385
32684    34.771284
32685    34.771389
32686    34.771393
32687    34.771397
32688    34.771401
32689    34.771405
32690    34.771409
32691    34.

経度および緯度のデータををdataの'lng'カラムよよび'lat'カラムに入れましょう。

In [12]:
data['lng'] = data['geometry'].centroid.x
data['lat'] = data['geometry'].centroid.y

/home/ksn/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/home/ksn/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


先にdata.describe()のところで、平均路線価の値に0が多いであろうことが確認されました。実際にどれくらいの数あるのか調べてみます。

In [13]:
data['平均路線価'].value_counts()

0       13962
105       188
97        185
110       184
121       179
107       179
102       177
112       174
125       174
117       172
114       171
104       169
115       169
106       167
103       164
120       163
108       162
113       161
116       158
100       158
123       157
122       156
99        155
96        155
131       154
95        153
92        153
93        152
91        151
128       149
        ...  
2473        1
2182        1
1031        1
399         1
439         1
471         1
487         1
1598        1
8803        1
631         1
663         1
679         1
727         1
791         1
3062        1
1111        1
2313        1
3174        1
1143        1
3302        1
1351        1
1527        1
1575        1
3718        1
3830        1
1847        1
4070        1
1903        1
2137        1
1967        1
Name: 平均路線価, Length: 960, dtype: int64

平均路線価が0のところのデータは表示する必要がないので、dataから取り除きます。

In [14]:
data = data[data['平均路線価'] != 0]

In [15]:
data.head()

,MESH_CODE,最高路線価,最低路線価,平均路線価,geometry,lng,lat
302,S110W00522,66,63,65,"POLYGON ((135.50334 34.58777, 135.50334 34.586...",135.502796,34.587320
303,S110W00521,66,60,63,"POLYGON ((135.50225 34.58777, 135.50225 34.586...",135.501706,34.587316
304,S110W01025,60,60,60,"POLYGON ((135.50116 34.58776, 135.50116 34.586...",135.500616,34.587312
305,S110E00516,22,22,22,"POLYGON ((135.50769 34.58869, 135.50770 34.587...",135.507151,34.588240
306,S110W00520,29,22,26,"POLYGON ((135.50660 34.58869, 135.50661 34.587...",135.506061,34.588235


In [16]:
data['平均路線価'].value_counts()

105     188
97      185
110     184
107     179
121     179
102     177
112     174
125     174
117     172
114     171
115     169
104     169
106     167
103     164
120     163
108     162
113     161
116     158
100     158
123     157
122     156
96      155
99      155
131     154
95      153
92      153
93      152
91      151
118     149
128     149
       ... 
4070      1
791       1
3062      1
1031      1
1111      1
3174      1
1143      1
3302      1
1351      1
1527      1
1575      1
3718      1
3830      1
1847      1
1903      1
1064      1
2313      1
2473      1
552       1
584       1
632       1
648       1
712       1
728       1
744       1
760       1
872       1
904       1
936       1
2137      1
Name: 平均路線価, Length: 959, dtype: int64

前処理が終わったので、pydeckを使って平均路線価の可視化を行います。

In [17]:
midpoint = (np.average(data['lat']), np.average(data['lng']))

r = pdk.Deck(
        map_style="mapbox://styles/mapbox/dark-v9",
        mapbox_key=MAPBOX_KEY,
        initial_view_state = {
            "latitude": midpoint[0],
            "longitude": midpoint[1],
            "zoom": 9,
            "pitch": 30,
          },
        layers = [
            pdk.Layer(
            'GridCellLayer',
            data = data[['平均路線価','lng','lat']],
            get_position = ['lng', 'lat'], #['longitude(経度)', 'latitude(緯度)']
            extruded = True,
            pickable = True,
            wireframe=True,
            get_elevation = '平均路線価',
            get_fill_color='[255, 255, 0]',
            get_line_color=[0,0,0],
            opacity = 0.1,
            auto_highlight=True,
            cell_size =  50,
            ),
        ],
    )

r.to_html("osakadeta.html")

'/home/ksn/Desktop/研究/研究資料一般/研究資料2020年度/PyConJP2020/tutorial/data3/osakadeta.html'

## 2. 平均路線価と自転車盗難数の可視化

市町村コードと緯度・経度の対応表を得るために、国土数値情報市区町村役場データを使います。

In [18]:
code_osaka = gpd.read_file('../data/osakadata/P34-14_27_GML/P34-14_27.shp', encoding='shift-jis')

In [19]:
code_osaka.head()

,P34_001,P34_002,P34_003,P34_004,geometry
0,27100,1,大阪市役所,大阪市北区中之島1-3-20,POINT (135.50205 34.69389)
1,27102,1,大阪市都島区役所,大阪市都島区中野町2-16-20,POINT (135.52809 34.70128)
2,27103,1,大阪市福島区役所,大阪市福島区大開1-8-1,POINT (135.47223 34.69236)
3,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304)
4,27106,1,大阪市西区役所,大阪市西区新町4-5-14,POINT (135.48611 34.67638)


In [20]:
code_osaka['緯度'] = code_osaka['geometry'].y
code_osaka['経度'] = code_osaka['geometry'].x

In [21]:
code_osaka.head()

,P34_001,P34_002,P34_003,P34_004,geometry,緯度,経度
0,27100,1,大阪市役所,大阪市北区中之島1-3-20,POINT (135.50205 34.69389),34.693891,135.502046
1,27102,1,大阪市都島区役所,大阪市都島区中野町2-16-20,POINT (135.52809 34.70128),34.701279,135.528090
2,27103,1,大阪市福島区役所,大阪市福島区大開1-8-1,POINT (135.47223 34.69236),34.692357,135.472232
3,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225
4,27106,1,大阪市西区役所,大阪市西区新町4-5-14,POINT (135.48611 34.67638),34.676384,135.486111


In [22]:
code_osaka = code_osaka.rename({'P34_001':'コード'}, axis=1)

In [23]:
code_osaka.head()

,コード,P34_002,P34_003,P34_004,geometry,緯度,経度
0,27100,1,大阪市役所,大阪市北区中之島1-3-20,POINT (135.50205 34.69389),34.693891,135.502046
1,27102,1,大阪市都島区役所,大阪市都島区中野町2-16-20,POINT (135.52809 34.70128),34.701279,135.528090
2,27103,1,大阪市福島区役所,大阪市福島区大開1-8-1,POINT (135.47223 34.69236),34.692357,135.472232
3,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225
4,27106,1,大阪市西区役所,大阪市西区新町4-5-14,POINT (135.48611 34.67638),34.676384,135.486111


次に、大阪の自転車盗難数に関するデータを読み込みんでデータフレームを作成します。このデータフレームは、先につくった市町村コードと緯度・経度の関係を表したデータフレーム(code_osaka)とマージすることになります。

大阪の自転車盗難数に関するデータは、大阪府警察犯罪オープンデータサイト (https://www.police.pref.osaka.lg.jp/seikatsu/9290.html) が出典です。

では、データを読み込んで解析を進めていきましょう。

In [24]:
hanzai_data = pd.read_csv('../data/osakadata/osaka_2018zitensyatou.csv', encoding='shift_jis')

In [25]:
hanzai_data.head()

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所の属性,被害者の年齢,被害者の職業,施錠関係
0,窃盗,自転車盗,西,NaN,271284.0,大阪府,大阪市中央区,千日前２丁目,2017-03-08,18,駐車（輪）場,法人・団体、被害者なし,法人・団体、被害者なし,施錠した
1,窃盗,自転車盗,西,松島交番,271063.0,大阪府,大阪市西区,立売堀３丁目,2018-05-12,15,４階建て以上共同住宅,50歳代,その他,施錠した
2,窃盗,自転車盗,西,松島交番,271063.0,大阪府,大阪市西区,江之子島１丁目,2018-03-18,21,４階建て以上共同住宅,70歳以上,その他,施錠した
3,窃盗,自転車盗,西,松島交番,271063.0,大阪府,大阪市西区,川口１丁目,2018-11-27,15,駐車（輪）場,60-64歳,その他,施錠した
4,窃盗,自転車盗,西,松島交番,271063.0,大阪府,大阪市西区,川口２丁目,2018-11-28,14,道路上,65-69歳,その他,施錠した


このデータから、各々の地域で発生した自転車盗難の回数を計算していきます。

市区町村コード（発生地）ごとにグループ化し、それぞれのコードを持ったデータの頻度（= 地区コード別の自転車盗難発生回数）をsizeメソッドを使って計算します。

In [26]:
grouped = hanzai_data.groupby('市区町村コード（発生地）')

In [27]:
grouped_size = grouped.size()

In [28]:
grouped_size.head()

市区町村コード（発生地）
271021.0    578
271039.0    324
271047.0    213
271063.0    500
271071.0    314
dtype: int64

SeriesをDataFrameにします。

In [29]:
grouped_size_df = pd.DataFrame(grouped_size)

In [30]:
grouped_size_df.head()

,0
市区町村コード（発生地）,
271021.0,578
271039.0,324
271047.0,213
271063.0,500
271071.0,314


インデックスが市区町村コード（発生地）になってしまっているので、インデックスをリセットします。

In [31]:
grouped_size_df.reset_index(inplace=True)

In [32]:
grouped_size_df.head()

,市区町村コード（発生地）,0
0,271021.0,578
1,271039.0,324
2,271047.0,213
3,271063.0,500
4,271071.0,314


自転車盗難件数に当たるカラムのカラム名が0となっていて分かりにくいので名前を「自転車盗難件数」に変えます。

In [33]:
grouped_size_df = grouped_size_df.rename({0:'自転車盗難件数'}, axis=1)

In [34]:
grouped_size_df.head()

,市区町村コード（発生地）,自転車盗難件数
0,271021.0,578
1,271039.0,324
2,271047.0,213
3,271063.0,500
4,271071.0,314


市区町村コード（発生地）カラムの値がfloatになっているので、文字列に変換します。

In [35]:
grouped_size_df['市区町村コード（発生地）'] = grouped_size_df['市区町村コード（発生地）'].astype(str)

市区町村コードは5桁なので(6桁目はチェックディジット)、5桁に直します。

In [36]:
grouped_size_df['市区町村コード（発生地）'] = grouped_size_df['市区町村コード（発生地）'].apply(lambda x: x[0:len(x)-3])

In [37]:
grouped_size_df.head()

,市区町村コード（発生地）,自転車盗難件数
0,27102,578
1,27103,324
2,27104,213
3,27106,500
4,27107,314


データ前処理の最後として、code_osakaというデータフレームとgrouped_size_dfというデータフレームをマージします。

In [38]:
osaka_df = code_osaka.merge(grouped_size_df, left_on='コード', right_on='市区町村コード（発生地）')

これで、特定の市区町村の緯度経度情報と自転車盗難件数の情報が統合されました。

In [39]:
osaka_df.head()

,コード,P34_002,P34_003,P34_004,geometry,緯度,経度,市区町村コード（発生地）,自転車盗難件数
0,27102,1,大阪市都島区役所,大阪市都島区中野町2-16-20,POINT (135.52809 34.70128),34.701279,135.528090,27102,578
1,27103,1,大阪市福島区役所,大阪市福島区大開1-8-1,POINT (135.47223 34.69236),34.692357,135.472232,27103,324
2,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213
3,27106,1,大阪市西区役所,大阪市西区新町4-5-14,POINT (135.48611 34.67638),34.676384,135.486111,27106,500
4,27107,1,大阪市港区役所,大阪市港区市岡1-15-25,POINT (135.46061 34.66392),34.663918,135.460611,27107,314


In [40]:
max(osaka_df['自転車盗難件数'])

1882

In [41]:
min(osaka_df['自転車盗難件数'])

1

前処理が終わったので、可視化を行います。
今回は2種類のレイヤーを重ねたものを可視化してみます。

In [42]:
midpoint = (np.average(osaka_df['緯度']), np.average(osaka_df['経度']))

INITIAL_VIEW_STATE = {
            "latitude": midpoint[0],
            "longitude": midpoint[1],
            "zoom": 9,
            "pitch": 30,
          }

layer1 = pdk.Layer(
            #"HexagonLayer",
            #'GridCellLayer',
            #"ColumnLayer",
            #"HeatmapLayer",
            "ScreenGridLayer",
            data = osaka_df[['自転車盗難件数','経度','緯度']],
            get_position = ['経度', '緯度'], #['longitude(経度)', 'latitude(緯度)']
            pickable = True,
            wireframe=True,
            get_weight='自転車盗難件数',
            opacity = 1.0,
            auto_highlight=True,
            cell_size =  200
            )

layer2 = pdk.Layer(
            #"HexagonLayer",
            'GridCellLayer',
            data = data[['平均路線価','lng','lat']],
            get_position = ['lng', 'lat'], #['longitude(経度)', 'latitude(緯度)']
            pickable = True,
            wireframe=True,
            get_elevation = '平均路線価',
            get_fill_color='[255,255,255]',
            get_line_color=[0, 0, 0],
            opacity = 0.1,
            auto_highlight=True,
            cell_size =  50,
            )


r = pdk.Deck(
        map_style="mapbox://styles/mapbox/dark-v9",
        mapbox_key=MAPBOX_KEY,
        initial_view_state = INITIAL_VIEW_STATE,
        layers = [layer1,layer2],
    )

r.to_html("osaka_rosenka_jitensha.html")

'/home/ksn/Desktop/研究/研究資料一般/研究資料2020年度/PyConJP2020/tutorial/data3/osaka_rosenka_jitensha.html'

問題点: 自転車盗難データの可視化は「大体」で行われており、正確なものではありません。「大体」の傾向を知りたいときにはこのような可視化を用いることも考えられますが、国土交通省国土数値情報を使うことでより正確な可視化を目指すことが出来ます。次のパートでは、より正確な可視化の実現のためのコードを紹介します。

## 3. 平均路線価の可視化と自転車盗難数データのより正確な可視化

データの読み込みを行います。

In [43]:
osaka_geo = gpd.read_file('../data/osakadata/N03-180101_27_GML/N03-18_27_180101.shp', encoding='shift-jis')

In [44]:
osaka_geo

,N03_001,N03_002,N03_003,N03_004,N03_007,geometry
0,大阪府,None,大阪市,都島区,27102,"POLYGON ((135.51481 34.72057, 135.51489 34.720..."
1,大阪府,None,大阪市,福島区,27103,"POLYGON ((135.46247 34.68740, 135.46253 34.687..."
2,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.34936 34.62007, 135.34568 34.617..."
3,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.39884 34.65724, 135.39877 34.657..."
4,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.39893 34.65737, 135.39892 34.657..."
5,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.40040 34.65959, 135.40039 34.659..."
6,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.40092 34.66036, 135.40078 34.660..."
7,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.37963 34.66092, 135.38073 34.660..."
8,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.42092 34.66441, 135.42087 34.664..."
9,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.40276 34.67308, 135.40278 34.673..."


In [45]:
osaka_geo['N03_007']

0      27102
1      27103
2      27104
3      27104
4      27104
5      27104
6      27104
7      27104
8      27104
9      27104
10     27104
11     27104
12     27106
13     27107
14     27107
15     27108
16     27108
17     27109
18     27111
19     27113
20     27114
21     27115
22     27116
23     27117
24     27118
25     27119
26     27120
27     27121
28     27122
29     27123
       ...  
113    27228
114    27229
115    27229
116    27230
117    27231
118    27232
119    27232
120    27232
121    27232
122    27301
123    27321
124    27322
125    27341
126    27361
127    27362
128    27362
129    27366
130    27366
131    27366
132    27366
133    27366
134    27366
135    27366
136    27366
137    27366
138    27381
139    27382
140    27382
141    27382
142    27383
Name: N03_007, Length: 143, dtype: object

In [46]:
osaka_geo.head()

,N03_001,N03_002,N03_003,N03_004,N03_007,geometry
0,大阪府,None,大阪市,都島区,27102,"POLYGON ((135.51481 34.72057, 135.51489 34.720..."
1,大阪府,None,大阪市,福島区,27103,"POLYGON ((135.46247 34.68740, 135.46253 34.687..."
2,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.34936 34.62007, 135.34568 34.617..."
3,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.39884 34.65724, 135.39877 34.657..."
4,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.39893 34.65737, 135.39892 34.657..."


In [47]:
osaka_df.head()

,コード,P34_002,P34_003,P34_004,geometry,緯度,経度,市区町村コード（発生地）,自転車盗難件数
0,27102,1,大阪市都島区役所,大阪市都島区中野町2-16-20,POINT (135.52809 34.70128),34.701279,135.528090,27102,578
1,27103,1,大阪市福島区役所,大阪市福島区大開1-8-1,POINT (135.47223 34.69236),34.692357,135.472232,27103,324
2,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213
3,27106,1,大阪市西区役所,大阪市西区新町4-5-14,POINT (135.48611 34.67638),34.676384,135.486111,27106,500
4,27107,1,大阪市港区役所,大阪市港区市岡1-15-25,POINT (135.46061 34.66392),34.663918,135.460611,27107,314


osaka_dfとosaka_geoをマージします。

In [48]:
osaka_seikaku_df = osaka_df.merge(osaka_geo, left_on='コード', right_on='N03_007')

In [49]:
osaka_seikaku_df.head()

,コード,P34_002,P34_003,P34_004,geometry_x,緯度,経度,市区町村コード（発生地）,自転車盗難件数,N03_001,N03_002,N03_003,N03_004,N03_007,geometry_y
0,27102,1,大阪市都島区役所,大阪市都島区中野町2-16-20,POINT (135.52809 34.70128),34.701279,135.528090,27102,578,大阪府,None,大阪市,都島区,27102,"POLYGON ((135.51481 34.72057, 135.51489 34.720..."
1,27103,1,大阪市福島区役所,大阪市福島区大開1-8-1,POINT (135.47223 34.69236),34.692357,135.472232,27103,324,大阪府,None,大阪市,福島区,27103,"POLYGON ((135.46247 34.68740, 135.46253 34.687..."
2,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.34936 34.62007, 135.34568 34.617..."
3,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.39884 34.65724, 135.39877 34.657..."
4,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213,大阪府,None,大阪市,此花区,27104,"POLYGON ((135.39893 34.65737, 135.39892 34.657..."


In [50]:
osaka_seikaku_df = osaka_seikaku_df.rename({'geometry_y':'geometry'}, axis=1)

pydeckに市区町村界のデータを渡すとき、Polygonではなくリストにしてから渡す必要があります。そのため、geometryカラムのPolygonをリストに変換します。

ちなみに、Polygonは.areaでPolygonの面積(ここでは、特定の区の面積)を計算することができる。

In [51]:
osaka_seikaku_df['geometry'][0].area

0.0005979953980768535

また、.exterior.coordsとすることで外周の緯度と経度のリストを得ることができます。

In [52]:
list(osaka_seikaku_df['geometry'][0].exterior.coords)[1:10]

[(135.51489444941933, 34.720739441072396),
 (135.5153100000565, 34.7215530541302),
 (135.515490557843, 34.72190666665938),
 (135.5155083401379, 34.7219416673741),
 (135.51561778223493, 34.7221558292282),
 (135.5156797284368, 34.72230944062824),
 (135.51584333310336, 34.722714721007605),
 (135.51601083003584, 34.72312916188139),
 (135.5160727762377, 34.723282775080065)]

しかし、MultiPolygonオブジェクトはexterior属性を持ちません。

In [53]:
#osaka_seikaku_df['geometry'] = osaka_seikaku_df['geometry'].apply(lambda x: list(x.exterior.coords))

そのため、条件分岐を行い、geometryカラムのデータがMultiPolygonのときはその中に含まれるPolygonごとに.exterior.coordsを使ってリストを得る必要があります。

In [54]:
def polygon_to_list(data):
    import shapely
    if type(data) == shapely.geometry.multipolygon.MultiPolygon:
        multi_num = len(data)
        geo_data = list()
        for i in range(multi_num):
            geo_list = list(data[i].exterior.coords)
            geo_data += geo_list
        return geo_data
    else:
        geo_data = list(data.exterior.coords)
        return geo_data

上でつくった関数を適用します。

In [55]:
osaka_seikaku_df['geometry'] = osaka_seikaku_df['geometry'].apply(polygon_to_list)

In [56]:
osaka_seikaku_df.head()

,コード,P34_002,P34_003,P34_004,geometry_x,緯度,経度,市区町村コード（発生地）,自転車盗難件数,N03_001,N03_002,N03_003,N03_004,N03_007,geometry
0,27102,1,大阪市都島区役所,大阪市都島区中野町2-16-20,POINT (135.52809 34.70128),34.701279,135.528090,27102,578,大阪府,None,大阪市,都島区,27102,"[(135.5148072241741, 34.72056889184091), (135...."
1,27103,1,大阪市福島区役所,大阪市福島区大開1-8-1,POINT (135.47223 34.69236),34.692357,135.472232,27103,324,大阪府,None,大阪市,福島区,27103,"[(135.4624702720854, 34.6873977749874), (135.4..."
2,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213,大阪府,None,大阪市,此花区,27104,"[(135.3493570039227, 34.62006963970214), (135...."
3,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213,大阪府,None,大阪市,此花区,27104,"[(135.39883763904584, 34.657239828555305), (13..."
4,27104,1,大阪市此花区役所,大阪市此花区春日出北1-8-4,POINT (135.45222 34.68304),34.683038,135.452225,27104,213,大阪府,None,大阪市,此花区,27104,"[(135.39892658919075, 34.65737358562251), (135..."


最後に、より正確な自転車盗難データと路線価データの可視化を行います。

In [65]:
midpoint = (np.average(osaka_seikaku_df['緯度']), np.average(osaka_seikaku_df['経度']))

INITIAL_VIEW_STATE = {
            "latitude": midpoint[0],
            "longitude": midpoint[1],
            "zoom": 9,
            "pitch": 30,
          }

layer1 = pdk.Layer(
            #"GeoJsonLayer",
            "PolygonLayer",
            data = osaka_seikaku_df[['自転車盗難件数','経度','緯度','geometry']],
            get_polygon='geometry',
            get_fill_color='[255, ( 1 - (自転車盗難件数 / 1882) ) * 255, 0, 100]',
            pickable=True,
            opacity = 0.1,
            )

layer2 = pdk.Layer(
            #"HexagonLayer",
            'GridCellLayer',
            data = data[['平均路線価','lng','lat']],
            get_position = ['lng', 'lat'], #['longitude(経度)', 'latitude(緯度)']
            radius = 100,
            extruded = True,
            pickable = True,
            wireframe=True,
            get_elevation = '平均路線価',
            get_fill_color='[255,255,255]',
            get_line_color=[0, 0, 0],
            opacity = 0.8,
            auto_highlight=True,
            cell_size =  50,
            )


r = pdk.Deck(
        map_style="mapbox://styles/mapbox/dark-v9",
        mapbox_key=MAPBOX_KEY,
        initial_view_state = INITIAL_VIEW_STATE,
        layers = [layer1,layer2],
    )
r.to_html("osaka_seikaku.html")

'/home/ksn/Desktop/研究/研究資料一般/研究資料2020年度/PyConJP2020/tutorial/data3/osaka_seikaku.html'

## 4. Streamlitでアプリケーション化

私のマシン上およびGAE上で動いているものを見ていただきます。

## 5. 課題（ハンズオン）

自分が調べてみたいデータを探し、そのデータに対して今日やったことをひと通りやってみてください。データを探すコツは、緯度と経度があるデータを探すことがまず一つ。あとは、市町村コードというコードがついていると、ESRI社が提供している全国市区町村界データと組み合わせることで階級区分図を作成することができます。